#1_環境準備

In [1]:
!pip install tiktoken # 費用見積に利用（トークン数から費用見積が可能）
!pip install openai # OpenAIのライブラリ内のAzureOpenAI機能を利用

#2_API、モデル、ベースデータセットの準備

In [2]:
# 環境変数の準備
# from google.colab import userdata
AZURE_OPENAI_ENDPOINT="https://hogen-chatbot-team2-us.openai.azure.com/"
# AZURE_OPENAI_API_KEY=userdata.get('AZURE_OPENAI_API_KEY')
AZURE_OPENAI_API_KEY="ff0ee84c7427430d96a5839a4dd3a23b"
AZURE_OPENAI_API_VERSION="2023-03-15-preview"

In [3]:
# apiクライアントの準備
from openai import AzureOpenAI
client = AzureOpenAI(
  azure_endpoint = AZURE_OPENAI_ENDPOINT,
  api_key = AZURE_OPENAI_API_KEY,
  api_version = AZURE_OPENAI_API_VERSION,
)

In [4]:
# Hugging Faceからデータセットの取得
import pandas as pd
import datasets
dataset = datasets.load_dataset("kunishou/databricks-dolly-15k-ja")
df = pd.DataFrame(dataset['train'])

/anaconda/envs/create_dataset_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/anaconda/envs/create_dataset_env/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
df

,output,input,index,category,instruction
0,ヴァージン・オーストラリア航空は、2000年8月31日にヴァージン・ブルー航空として、2機の...,ヴァージン・オーストラリア航空（Virgin Australia Airlines Pty ...,0,closed_qa,ヴァージン・オーストラリア航空はいつから運航を開始したのですか？
1,イコクエイラクブカ,,1,classification,魚の種類はどっち？イコクエイラクブカとロープ
2,ラクダは、長時間にわたってエネルギーと水分で満たされた状態を保つために、腰の脂肪を利用しています。,,2,open_qa,ラクダはなぜ水なしで長く生きられるのか？
3,三女の名前はアリス,,3,open_qa,アリスの両親には3人の娘がいる：エイミー、ジェシー、そして三女の名前は？
4,小森田友明は1981年7月10日に生まれました。,小森田は1981年7月10日、熊本県に生まれる。高校卒業後、2000年にJ1リーグのアビスパ...,4,closed_qa,小森田友明はいつ生まれたの？
...,...,...,...,...,...
15010,変化を受け入れて、違いを見る,,15010,brainstorming,変更を受け入れるにはどうしたらよいですか
15011,レーザーは、電磁波源から光を放出する装置である。 最初のレーザーは、チャールズ・H・タウン...,レーザーは、電磁波の誘導放出に基づく光増幅の過程で光を放出する装置である。レーザーという言葉...,15011,summarization,レーザーとは何か、誰が作ったのか？
15012,ロードバイクはアスファルトやセメントの上を走ることを想定して作られており、細いタイヤが装着さ...,,15012,open_qa,ロードバイクとマウンテンバイクの違いは何ですか？
15013,不動産投資家は、ビジネスを展開する市場や場所に関する競争力を高めるために、正確で精度の高いロ...,,15013,general_qa,不動産投資業界において、GISはどのように役立っているのでしょうか。


#3_料金見積

#4_データセットの作成

In [6]:
# ベースのデータセット内のoutput(アシスタントの返答データ)を関西弁に変換
# gpt_outputsというリストを初期化し、countを0に設定
gpt_outputs = []
error_idx = []
s_num = 0
e_num = 1000

# df["output"]の最初の10件の文章に対してループを実行
for i, st in enumerate(df["output"][s_num:e_num]):
    try:
        # GPT-4o-miniモデルを使って、文章を関西弁に変換するリクエストを送信
        res = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "user", "content": """
                以下の文章をこてこての関西弁に変換し、その結果のみを回答してください。
                意味が伝わればよいので、方言の強度を最高にして現地の人のようにフランクに回答してください。ただし、知らない単語はそのまま出力してください。
                """+st}]
        )
        # 変換された関西弁のテキストをgpt_outputsに追加
        gpt_outputs.append(res.choices[0].message.content)
    except:
        # エラーが発生した場合、countをインクリメントし、Noneをgpt_outputsに追加
        gpt_outputs.append(None)
        error_idx.append(i)
        continue

In [7]:
gpt_outputs

['ヴァージン・オーストラリア航空ちゅうのは、2000年8月31日にヴァージン・ブルー航空ちゅう名前で、2機の飛行機で単一路線の運航始めたんやで。',
 'イコクエイラクブカやで！',
 'ラクダはな、長時間ずっとエネルギーと水分で満たされた状態を保つために、腰の脂肪を利用しとんねん。',
 '三女の名前はアリスやで！',
 '小森田友明は1981年7月10日に生まれたんやで。',
 'あかんで。ステイルメイトっちゅうんは、引き分けた状態のことや。どっちがより多くの駒を捕まえたか、または優勢かは関係あらへんで。',
 'ロラパルーズっちゅうんは、イリノイ州シカゴのグラントパークで毎年開催される音楽フェスやねん。1991年にジェーンズ・アディクションのボーカルのペリー・ファレルのお別れツアーから始まったんや。このフェスには、オルタナティブ・ロック、ヘビーメタル、パンクロック、ヒップホップ、エレクトロニック・ダンス・ミュージックなんか、いろんなジャンルの音楽があんねん。毎年40万人も観客が集まって、チケットは毎年すぐに売り切れや。主な出演者はレッド・ホット・チリ・ペッパーズ、チャンス・ザ・ラッパー、メタリカ、レディ・ゲイジとかやで。ロラパルーザは、世界最大かつ最も象徴的な音楽フェスの1つで、シカゴの定番イベントなんや。',
 'ジョン・デイヴィソン・ロックフェラーやで！',
 'ずっとスマホいじってんのがよぉないからやねん。',
 'ジョン・モーゼス・ブローニングっちゅうのは、近代銃器の設計者としてめっちゃ有名な人の一人や。13歳のときに親父の店で銃器の製作始めて、24歳のときに最初の特許取ったんやで。\n\nそれから、単発式、レバーアクション式、ポンプアクション式のライフルやショットガンを発明・改良しまくったんや。\n\n今じゃ、彼はM1911ピストル、ブローニング自動小銃、オート5散弾銃で一番知られてて、これらは全部オリジナルデザインかマイナーチェンジされた状態で今でも作られとるんや。M1911ピストルやハイパワーピストルのデザインは、今や世界で一番複製された銃器のひとつになっとるわ。',
 'トーマス・ジェファーソン（1743年4月13日～1826年7月4日）は、アメリカの政治家、外交官、弁護士、建築家、哲学者、建国の父やねん。1801年から1809年までアメリカ

#5_データセットの出力・保存

In [8]:
import csv, datetime
from datetime import datetime, timedelta, timezone

# タイムゾーンの生成
JST = timezone(timedelta(hours=+9), 'JST')
now = datetime.now(JST).strftime('%m%d_%H%M%S')

# 出力範囲の設定（outputを変更した列のみ）
df_to_csv = df[s_num:e_num].copy()
# 出力範囲内のoutputを関西弁版に置換
df_to_csv['output'] = gpt_outputs
# 置換後のデータセットをcsv出力
df_to_csv.to_csv('./datasets/dataset_kansaiben_' + str(s_num)+'-'+str(e_num) +'_'+now +'.csv', index=False)

print("csv出力完了")

csv出力完了


In [9]:
df_to_csv

,output,input,index,category,instruction
0,ヴァージン・オーストラリア航空ちゅうのは、2000年8月31日にヴァージン・ブルー航空ちゅう...,ヴァージン・オーストラリア航空（Virgin Australia Airlines Pty ...,0,closed_qa,ヴァージン・オーストラリア航空はいつから運航を開始したのですか？
1,イコクエイラクブカやで！,,1,classification,魚の種類はどっち？イコクエイラクブカとロープ
2,ラクダはな、長時間ずっとエネルギーと水分で満たされた状態を保つために、腰の脂肪を利用しとんねん。,,2,open_qa,ラクダはなぜ水なしで長く生きられるのか？
3,三女の名前はアリスやで！,,3,open_qa,アリスの両親には3人の娘がいる：エイミー、ジェシー、そして三女の名前は？
4,小森田友明は1981年7月10日に生まれたんやで。,小森田は1981年7月10日、熊本県に生まれる。高校卒業後、2000年にJ1リーグのアビスパ...,4,closed_qa,小森田友明はいつ生まれたの？
...,...,...,...,...,...
995,わいは海岸に座ってるで！ \n観波 \n一再ならず,,995,creative_writing,海岸に座って波を見ている俳句を作ろう。
996,ジャン＝バティスト・セイの『政治経済学論』ちゅうやつやな、または『富の生産、分配および消費』...,社会科学としての黎明期、経済学は、ジャン＝バティスト・セイが『政治経済学論考』あるいは『富の...,996,information_extraction,提供された文章から、初期の経済学者の名前と発表された作品を抽出しなさい。名前を特定し、次にダ...
997,国名：スウェーデン、フランス、インド、ポルトガル \n都市名：パリ、バリ、ロッテルダム、セ...,,997,classification,次の各項目を国または都市のいずれかに分類しなさい：スウェーデン、フランス、パリ、バリ、ロッテ...
998,上海マグレブ（中国・上海）やな！,,998,open_qa,世界最速の列車は何ですか？
